In [111]:
import numpy as np
import pandas as pd

In [112]:
class Caminante():
    def __init__(self,inicial,estado,n_caminante):
        self.inicial = inicial.clonar()
        self.mutacion = estado.clonar()
        self.n_caminante = n_caminante
        self.norma = self.mutacion.norma()
        self.vr = 0
    def clonar(self):
        sample = 0
        other = Caminante(self.inicial,self.mutacion,self.n_caminante)
        return other
    def mutar(self,alfa):
        self.mutacion.mutar(alfa = alfa)
        self.norma = self.mutacion.norma()
    def evaluar(self,sample):
        self.reward = self.mutacion.evaluar(sample)

In [113]:
class binary():
    def __init__(self,capas=1,nodos=1):
        self.capas = capas
        self.nodos = nodos
        self.forma = (self.capas,self.nodos)
    def inicializar(self):
        self.red = []
        for capa in range(self.capas):
            w = np.random.normal(size = (self.nodos,self.nodos))
            b = np.random.normal(size = self.nodos)
            self.red.append([w,b])
        w_f = np.random.normal(size = self.nodos)
        b_f = np.random.normal(size = 1)
        self.red.append([w_f,b_f])
    def mutar(self,alfa=0.1):
        red = []
        for r in self.red:
            r0 = r[0]+alfa*np.random.normal(size = r[0].shape)
            r1 = r[1]+alfa*np.random.normal(size = r[1].shape)
            red.append([r0,r1])
        hijo = binary(self.capas,self.nodos)
        hijo.inicializar()
        hijo.red = red[:]
        return hijo
    def cruzar(self,other):
        if self.forma == other.forma:
            hijo = binary(self.capas,self.nodos)
            hijo.inicializar()
            red = []
            for i in range(len(self.red)):
                r0 = (self.red[i][0] + other.red[i][0])/2.0
                r1 = (self.red[i][1] + other.red[i][1])/2.0
                red.append([r0,r1])
            hijo.red = red[:]
            return hijo
        else:
            return None
    def clonar(self):
        hijo = binary(self.capas,self.nodos)
        hijo.inicializar()
        red = []
        for i in range(len(self.red)):
            r0 = self.red[i][0][:]
            r1 = self.red[i][1][:]
            red.append([r0,r1])
        hijo.red = red[:]
        return hijo
    def distancia(self,other):
        if self.forma == other.forma:
            distancia = 0
            for i in range(len(self.red)):
                r0 = (self.red[i][0] - other.red[i][0])**2
                r1 = (self.red[i][1] - other.red[i][1])**2
                distancia = distancia + r0.sum().sum()+r1.sum().sum()
                distancia = np.sqrt(distancia)
            return distancia
        else:
            return None
    def norma(self):
        if True:
            distancia = 0
            for i in range(len(self.red)):
                r0 = (self.red[i][0])**2
                r1 = (self.red[i][1])**2
                distancia = distancia + r0.sum().sum()+r1.sum().sum()
                distancia = np.sqrt(distancia)
            return distancia
        else:
            return None
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def predecir(self,entrada):
        for r in self.red:
            entrada = np.dot(entrada,r[0])
            entrada = entrada + r[1]
            entrada = self.sigmoid(entrada)
        entrada = entrada.round()
        entrada = int(entrada)
        return entrada
    def evaluar(self,muestra):
        target = muestra.target
        u = muestra.columns.tolist()
        v = []
        for i in u:
            if i != 'target':
                v.append(i)
        entrada = muestra.loc[:,v].values.tolist()
        resultado = target.tolist()
        suma = 0
        cuenta = 0
        for i in range(len(entrada)):
            u = self.predecir(entrada[i])
            v = resultado[i]
            if u == v:
                suma += 1
            cuenta += 1
        u = float(suma)/float(cuenta)
        return u  
    def razon(self,other,n_pasos):
        if self.forma == other.forma:
            hijo = binary(self.capas,self.nodos)
            hijo.inicializar()
            red = []
            for i in range(len(self.red)):
                r0 = (-self.red[i][0] + other.red[i][0])/(n_pasos)+self.red[i][0]
                r1 = (-self.red[i][1] + other.red[i][1])/(n_pasos)+self.red[i][1]
                red.append([r0,r1])
            self.red = red
                
            

In [114]:
def crear_traza(individuo,n_pasos,muestra,n_sample,alfa=0.1):
    ''' Devuelve el mejor individuo, el individuo promedio y el ranking
    tanto del mejor individuo como del promedio'''
    prediccion_traza = 0
    individuo_ = individuo.clonar()
    mejor_individuo = individuo.clonar()
    mejor_prediccion = mejor_individuo.evaluar(muestra.sample(n_sample))
    individuos = []
    pesos = []
    for i in range(n_pasos):
        hijo = individuo.mutar(alfa)
        individuos.append(hijo)
        traza = hijo.clonar()
        prediccion = hijo.evaluar(muestra.sample(n_sample))
        pesos.append(prediccion)
        if prediccion > mejor_prediccion:
            mejor_prediccion = prediccion
            mejor_individuo = hijo.clonar()
        prediccion_traza += prediccion
        individuo = hijo.clonar()      
    traza=promediar(individuos,pesos)
    red = []
    for r in range(len(traza.red)):
        r0=traza.red[r][0]+alfa*(individuo_.red[r][0]-traza.red[r][0])/(individuo_.norma()+traza.norma())
        r1=traza.red[r][1]+alfa*(individuo_.red[r][1]-traza.red[r][1])/(individuo_.norma()+traza.norma())
        red.append([r0,r1])
    traza.red = red
    prediccion_traza = prediccion_traza/float(n_pasos)
    return mejor_prediccion, mejor_individuo, prediccion_traza, traza

In [116]:
titanic = pd.read_csv('train.csv')
titanic.groupby('Embarked').size()
titanic = pd.read_csv('train.csv')
titanic = titanic[['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
titanic.loc[titanic.Embarked=='C','Embarked']=-1
titanic.loc[titanic.Embarked=='Q','Embarked']=0
titanic.loc[titanic.Embarked=='S','Embarked']=1
titanic['target'] = titanic.Survived
titanic['Pclass_1']=np.where(titanic.Pclass==1,1,0)
titanic['Pclass_2']=np.where(titanic.Pclass==2,1,0)
titanic['Pclass_3']=np.where(titanic.Pclass==3,1,0)
titanic['Sex']=np.where(titanic.Sex=='male',1,0)
titanic = titanic.fillna(titanic.mean())
titanic[['Age']] = titanic[['Age']]/titanic[['Age']].mean()
titanic[['Fare']] = titanic[['Fare']]/titanic[['Fare']].mean()
del titanic['Pclass'],titanic['Survived']


In [117]:
titanic_train = titanic[titanic.target==0].sample(250)\
                .append(titanic[titanic.target==1].sample(250))
titanic_test = titanic.loc[~titanic.index.isin(titanic_train.index),:]

In [195]:
def promediar(individuos):
    individuo_0 = individuos[0].clonar()
    red = individuo_0.red[:]
    for r in range(len(red)):
        r0 = red[r][0]
        r1 = red[r][1]
        red[r][0]= -r0+red[r][0]
        red[r][1]= -r1+red[r][1]
        
    for individuo in individuos:
        for r in range(len(red)):
            r0 = (individuo.red[r][0][:])
            r1 = (individuo.red[r][1][:])
            red[r][0]=r0+red[r][0]
            red[r][1]=r1+red[r][1]
    n_red = []
    for r in range(len(red)):
        red[r][0]=red[r][0]/float(len(individuos))
        red[r][1]=red[r][1]/float(len(individuos))
        n_red.append([red[r][0],red[r][1]])
    individuo_0.red = n_red
    individuo_0.red = red
    return individuo_0

In [230]:
individuo = binary(2,9)
individuo.inicializar()

In [233]:

n= 1000
m = 10 # longitud traza
walkers = 10
alfa = 10
alpha = alfa*np.exp(np.log(10**-5)/n)
n_sample = 50
print(individuo.evaluar(titanic_train))

for j in range(n):
    caminantes = []
    for i in range(walkers):
        ind = individuo.mutar()
        caminante = Caminante(ind,ind.mutar(),n_caminante=i)
        caminante.mutar(alfa=alpha)
        caminante.evaluar(titanic_test.sample(n_sample))
        caminantes.append(caminante)


    for i in range(m):
        # crear walkers

        for caminante in caminantes:
            select = np.random.choice(caminantes)
            while select is caminante:
                select = np.random.choice(caminantes)
            d = caminante.mutacion.distancia(select.mutacion)
            caminante.vr = caminante.reward*d
                
        nuevos_caminantes = []
        for caminante in caminantes:
            select = np.random.choice(caminantes)
            if caminante.vr == 0:
                prob = 1
            elif caminante.vr > select.vr:
                prob = 0
            else:
                prob = (select.vr -caminante.vr)/select.vr
            if np.random.random()<prob:
                nuevo_caminante = select.clonar()
            else:
                nuevo_caminante = caminante.clonar()
            nuevos_caminantes.append(nuevo_caminante)
        caminantes = []
        for caminante in nuevos_caminantes:
            caminante.mutar(alpha)
            caminante.evaluar(titanic_train.sample(n_sample))
            caminantes.append(caminante)
    individuos = []
    for caminante in caminantes:
        individuos.append(caminante.inicial)
    individuo = promediar(individuos)
    print(individuo.evaluar(titanic_train),individuo.norma())


0.68
0.696 4.066573143669235
0.698 4.087919890581039
0.674 4.0752820748024
0.672 4.090291201928908
0.698 4.107713925645453
0.712 4.091360591639492
0.714 4.116032160093218
0.708 4.160453890881777
0.656 4.147729623338662
0.694 4.159871337705422
0.714 4.167417314082764
0.714 4.232386718119613
0.718 4.264624864754132
0.716 4.259002644865554
0.712 4.265562049714303
0.708 4.225833251891936
0.654 4.194695827528937
0.652 4.2499482779774
0.708 4.21414177549145
0.71 4.250991309545041
0.706 4.286600921544984
0.718 4.30232721452091
0.696 4.335032850868403
0.726 4.316447327133037
0.72 4.401134020651724
0.716 4.364188284082966
0.712 4.322988914351526
0.73 4.332902062083093
0.728 4.370518709197018
0.714 4.395394706449597
0.706 4.403861991473334
0.71 4.372955184802164
0.7 4.377098323722062
0.716 4.338698618976423
0.704 4.340028760715319
0.708 4.330670747742871
0.708 4.342992796112422
0.714 4.394390241301879
0.712 4.44064916807426
0.71 4.460764239485982
0.718 4.423385296785142
0.716 4.417193658423846
0

KeyboardInterrupt: 

In [ ]:
caminante.inicial

In [ ]:
best_model = mejor_traza


In [ ]:
best_model.evaluar(titanic_test)

In [ ]:
titanic = pd.read_csv('test.csv')
titanic = titanic[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
titanic.loc[titanic.Embarked=='C','Embarked']=-1
titanic.loc[titanic.Embarked=='Q','Embarked']=0
titanic.loc[titanic.Embarked=='S','Embarked']=1
titanic['Pclass_1']=np.where(titanic.Pclass==1,1,0)
titanic['Pclass_2']=np.where(titanic.Pclass==2,1,0)
titanic['Pclass_3']=np.where(titanic.Pclass==3,1,0)
titanic['Sex']=np.where(titanic.Sex=='male',1,0)
titanic = titanic.fillna(titanic.mean())
titanic[['Age']] = titanic[['Age']]/titanic[['Age']].mean()
titanic[['Fare']] = titanic[['Fare']]/titanic[['Fare']].mean()
del titanic['Pclass']

In [ ]:
scores = []
for index, row in titanic.iterrows():
    scores.append(best_model.predecir(row.values))


In [ ]:
resultado = pd.read_csv('test.csv')
resultado['Survived'] = scores
resultado.Survived = resultado.Survived.round().astype(int)
resultado[['PassengerId','Survived']].to_csv('result.csv',index=False)